In [504]:
import time
import numpy as np
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
plt.style.use("seaborn-dark")
%matplotlib inline

In [505]:
def read_dataset(feature_file, label_file):
    '''Read dataset in *.csv to dataframe in pandas'''
    df_X = pd.read_csv(feature_file)
    df_y = pd.read_csv(label_file)
    X = df_X.values # convert values in dataframe to numpy array (feature)
    y = df_y.values # convert values in dataframe to numpy array (label)
    return X, y

In [506]:
def normalize_features(X_train, X_test):
    from sklearn.preprocessing import StandardScaler # import library 
    scaler = StandardScaler() # call an object function
    scaler.fit(X_train)   # calculate mean, std in X_train  (x-u)/s
    X_train_norm = scaler.transform(X_train)  # apply normalization on X_train
    X_test_norm = scaler.transform(X_test)    # apply normalization on X_test
    return X_train_norm, X_test_norm



In [507]:
def pcc(y_pred, y_test):
   '''Pearson correlation coefficient measures the linear relationship between two datasets. '''
   from scipy import stats
   a = y_test.ravel()
   b = y_pred.ravel()
   pcc = stats.pearsonr(a,b)[0]
   return pcc

def Accuracy(ypred, yexact):
   p = np.array(ypred == yexact, dtype = int)
   return np.sum(p)/float(len(yexact)) 


In [508]:
def RMSE(ypred, yexact):
    '''Root Mean Square Error'''
    return np.sqrt(np.sum((ypred - yexact)**2)/ ypred.shape[0])

^ Definition of important functions

In [509]:
X_train, y_train = read_dataset('Iris_X_train.csv', 'Iris_y_train.csv')
X_test, y_test = read_dataset('Iris_X_test.csv', 'Iris_y_test.csv')
X_train_norm, X_test_norm = normalize_features(X_train, X_test)

^ Importing Iris

In [510]:
print(X_train_norm.shape)
print(X_test_norm.shape)
print(y_train.shape)
print(y_test.shape)

(112, 2)
(38, 2)
(112, 1)
(38, 1)


Let $N$ denote the number of elements, $D$ the number of features. Then $X$ has shape $(N,D)$, $W$ has shape $(D,1)$, and we have:

\begin{align}
    \hat{y}  &= \frac{1}{1-e^{-c^Tx}}\\ 
    \text{let } z   &= c^Tx \\
                   &= c_0 + c_1x_1 + c_2x_2 \\
    \hat{y}      &= (1-e^{-z})^{-1} \\
    L &= -\frac{1}{M }\sum_{i = 1}^{M} (y^{(i)}\log(\hat{y}) + (1-y^{(i)})\log(1-\hat{y}) \\
    %\dfrac{\partial{L}}{\partial{W}}  &= \dfrac{\partial{L}}{\partial{\hat{y}}} \dfrac{\partial{\hat{y}}}{\partial{W}} = \frac{1}{N} X^T (\hat{y}-y))\\
    %\dfrac{\partial{L}}{\partial{b}}  &= \dfrac{\partial{L}}{\partial{\hat{y}}} \dfrac{\partial{\hat{y}}}{\partial{b}} = \sum_{i}\frac{1}{N}(\hat{y}_i-y_i) \\
  %  W                                 &:= W - lr*\dfrac{\partial{L}}{\partial{W}}\\
  %  b                                 &:= b - lr *\dfrac{\partial{L}}{\partial{b}}
\end{align}

Then, we need to calculate the partial derivatives with respect to $c_0, c_1$, and $c_2$. Notice that 
$$ \dfrac{\partial{z}}{\partial{c_i}} = x_i$$ where $x_0 = 0$.
Then, calculating 

\begin{align}
    \dfrac{\partial{\hat{y}}}{\partial{z}}  &= (-1)(1+e^{-z})^{-2})e^{-z}(-1) \\
    &= (1+e^{-z})^{-2})e^{-z} \\
    &= \frac{e^{-z}}{(1+e^{-z})^2} \\
    &= \frac{e^{-z}}{(1+e^{-z})} * \frac{1}{(1+e^{-z})}\\
    &= \frac{e^{-z}}{(1+e^{-z})} * \hat{y} \\
    &= \frac{-1+1+ e^{-z}}{1+e^{-z}} * \hat{y} \\
    &=(\frac{1}{1+e^{-z}} + \frac{-(1 +e^{-z})}{1+e^{-z}} )* \hat{y} \\
    &= (\hat{y} - 1)\hat{y}\\
\end{align}

And using that, we can find
\begin{align}
     \dfrac{\partial{L}}{\partial{c_i}} &= -\sum_{i=1}^{M} (\frac{y^{(i)}}{\hat{y}} \cdot \dfrac{\partial{\hat{y}}}{\partial{z}} \cdot  \dfrac{\partial{z}}{\partial{c_i}} - \frac{1-y^{(i)}}{1-\hat{y}} \cdot  \dfrac{\partial{\hat{y}}}{\partial{z}} \cdot  \dfrac{\partial{z}}{\partial{c_i}} ) \\
     &= -\sum_{i=1}^{M}  \dfrac{\partial{z}}{\partial{c_i}} (\frac{y^{(i)}}{\hat{y}} \cdot \dfrac{\partial{\hat{y}}}{\partial{z}}  - \frac{1-y^{(i)}}{1-\hat{y}} \cdot  \dfrac{\partial{\hat{y}}}{\partial{z}}) \\
     &= -\sum_{i=1}^{M}  \dfrac{\partial{z}}{\partial{c_i}} (\frac{y^{(i)}}{\hat{y}} \cdot (\hat{y} - 1)\hat{y}  - \frac{1-y^{(i)}}{1-\hat{y}} \cdot  (\hat{y} - 1)\hat{y}) \\
      &= -\sum_{i=1}^{M}  \dfrac{\partial{z}}{\partial{c_i}} (y^{(i)}(1-\hat{y})-(1-y^{(i)})\hat{y}) \\
     &= -\sum_{i=1}^{M}  \dfrac{\partial{z}}{\partial{c_i}} (y^{(i)}-\hat{y})
\end{align}

Then
\begin{align}
    \dfrac{\partial{L}}{\partial{c_1}} &= -\sum_{i=1}^{M}  \dfrac{\partial{z}}{\partial{c_i}} (y^{(i)}-\hat{y}) \\
                                      &= -\sum_{i=1}^M   0
\end{align}

In [511]:
class Logistic_Regression:
    def __init__(self, X, y, lr = 0.01):
        self.X1 = X[:,0]
        self.X2 = X[:,1]
        self.y = y
        self.lr = lr
        self.c0 = 0
        self.c1 = 0
        self.c2 = 0

    def forward(self):
        self.y_hat = 1.0 / (1+np.exp(-(self.c0 +(self.c1 * (self.X1) ) + (self.c2 * (self.X2)) ))).reshape(-1,1)
        # print(self.y_hat.shape)
        # print(self.y.shape)
        
    def gradientDescent(self):
        d = (self.y_hat-self.y) / self.X1.shape[0]
        dc1 = np.sum(d * (self.X1).T)
        dc0 = np.sum(d)
        dc2 = np.sum(d * (self.X2).T)
        self.c0 = self.c0 - self.lr * dc0
        self.c1 = self.c1 - self.lr * dc1
        self.c2 = self.c2 - self.lr * dc2
        
    def lossfunction(self):
        self.forward()
        self.loss =  -(1/self.X1.shape[0]) * np.sum((self.y) * np.log(self.y_hat) + (1-self.y) * np.log(1-self.y_hat))
        
    def predict(self, X_test):
        z = self.c0 + (self.c1 * (X_test[:,0]) ) + (self.c2 * (X_test[:,1]) )
        y_hat_test = 1.0 / (1+np.exp(-z)).ravel()

        ypred = np.zeros(X_test.shape[0],  dtype=int)
        for i in range(X_test.shape[0]):
            if y_hat_test[i] >= 0.5:
                ypred[i] = 1
            else:
                ypred[i] = 0
        return ypred


In [512]:
# myLR = Logistic_Regression(X_train_norm, y_train, 0.1)
# myLR.forward()
# myLR.gradientDescent()

In [513]:
# print(myLR.c0)
# print(myLR.c1)
# print(myLR.c2)

In [514]:
myLR = Logistic_Regression(X_train_norm, y_train, 0.01)
epoch_num = 2000
for i in range(epoch_num):
    myLR.forward()
    myLR.gradientDescent()
    myLR.lossfunction()
    if ((i+1)%5 == 0):
        print('epoch = %d, current loss = %.5f'%(i+1, myLR.loss))



epoch = 5, current loss = 0.69201
epoch = 10, current loss = 0.69090
epoch = 15, current loss = 0.68981
epoch = 20, current loss = 0.68876
epoch = 25, current loss = 0.68773
epoch = 30, current loss = 0.68672
epoch = 35, current loss = 0.68574
epoch = 40, current loss = 0.68479
epoch = 45, current loss = 0.68385
epoch = 50, current loss = 0.68294
epoch = 55, current loss = 0.68206
epoch = 60, current loss = 0.68119
epoch = 65, current loss = 0.68035
epoch = 70, current loss = 0.67952
epoch = 75, current loss = 0.67872
epoch = 80, current loss = 0.67794
epoch = 85, current loss = 0.67718
epoch = 90, current loss = 0.67643
epoch = 95, current loss = 0.67570
epoch = 100, current loss = 0.67500
epoch = 105, current loss = 0.67431
epoch = 110, current loss = 0.67363
epoch = 115, current loss = 0.67297
epoch = 120, current loss = 0.67233
epoch = 125, current loss = 0.67171
epoch = 130, current loss = 0.67110
epoch = 135, current loss = 0.67050
epoch = 140, current loss = 0.66992
epoch = 145,

In [515]:
y_pred = myLR.predict(X_test_norm)
print(y_pred.shape)
print(y_test.shape)
print('ACC is', Accuracy(y_pred,y_test.ravel()))

(38,)
(38, 1)
ACC is 0.7105263157894737


This approach didn't seem to be working for some reason, and I am unfamiliar with object classes, so I decided to try it without those:
